In [2]:
 # Full working dashboard code (paste into a Jupyter cell)
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Change these to match your CRUD module/class if different (based on your posted code)
from CRUD_Python_Module import AnimalShelter

# DB credentials and connect
username = "aacuser"
password = "Test1234!"
db = AnimalShelter(username, password)

# read all documents initially
records = list(db.read({}))            # expect list of dicts
df = pd.DataFrame.from_records(records)

# drop MongoDB _id if present
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# --- Assumed column names in the Austin Animal Center dataset ---
# If your CSV/DB uses different names, change these variable names below.
COL_BREED = 'breed'                         # breed description
COL_NAME = 'name'                           # animal name
COL_AGE_WEEKS = 'age_upon_outcome_in_weeks' # numeric weeks
COL_SEX = 'sex_upon_outcome'                # e.g. 'Intact Male', 'Intact Female', etc.
COL_LAT = 'latitude'
COL_LONG = 'longitude'

# Ensure our df has those columns; if not, create empty ones to avoid crashes
for c in [COL_BREED, COL_NAME, COL_AGE_WEEKS, COL_SEX, COL_LAT, COL_LONG]:
    if c not in df.columns:
        df[c] = np.nan

# Build app
app = JupyterDash(__name__)

# Logo (replace path if necessary). Developer uploaded: /mnt/data/Grazioso Salvare Logo.png
logo_path = 'Grazioso Salvare Logo.png'
if os.path.exists(logo_path):
    encoded_logo = base64.b64encode(open(logo_path, 'rb').read()).decode()
    logo_img = html.Img(src=f'data:image/png;base64,{encoded_logo}', style={'height':'120px'})
else:
    logo_img = html.Div("Grazioso Salvare Logo not found", style={'color':'red'})

# Unique identifier (replace with your name)
unique_id = html.Div("Created by: Mark Pemberton", style={'fontSize': '14px', 'marginTop':'8px'})

# Radio options for filters
filter_radio = dcc.RadioItems(
    id='filter-type',
    options=[
        {'label': 'Water Rescue', 'value': 'water'},
        {'label': 'Mountain / Wilderness Rescue', 'value': 'mountain'},
        {'label': 'Disaster / Individual Tracking', 'value': 'disaster'},
        {'label': 'Reset (All Dogs)', 'value': 'reset'}
    ],
    value='reset',
    labelStyle={'display': 'block'}
)

# DataTable setup (pagination, sorting, selectable rows)
table = dash_table.DataTable(
    id='datatable-id',
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
    data=df.to_dict('records'),
    page_size=10,
    sort_action='native',
    filter_action='native',
    row_selectable='single',
    selected_rows=[],
    style_table={'overflowX': 'auto'},
)

# layout
app.layout = html.Div([
    html.Div([logo_img, unique_id], style={'display':'flex', 'alignItems':'center', 'gap':'20px'}),
    html.Center(html.B(html.H1('Grazioso Salvare — Austin Animal Center Dashboard'))),
    html.Hr(),

    html.Div([
        html.Div([
            html.H4("Filter by Rescue Type"),
            filter_radio,
            html.Br(),
            html.Button("Apply Filter", id='apply-filter', n_clicks=0)
        ], style={'width':'25%', 'padding':'10px', 'borderRight':'1px solid #ddd'}),

        html.Div([
            table
        ], style={'width':'75%', 'padding':'10px'})
    ], style={'display':'flex'}),

    html.Hr(),

    # charts / map area
    html.Div(className='row', style={'display' : 'flex', 'gap':'20px'},
             children=[
                 html.Div(id='graph-id', className='col s12 m6', style={'flex':1}),
                 html.Div(id='map-id', className='col s12 m6', style={'flex':1})
             ])
])

###############################
# Filtering logic (controller)
###############################

# Rescue-type preferences (per Dashboard Specifications Document)
WATER_BREEDS = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
MOUNTAIN_BREEDS = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]
DISASTER_BREEDS = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]

def build_query_for_filter(filter_type):
    """Return MongoDB query dict for the given filter type."""
    if filter_type == 'water':
        query = {
            COL_BREED: {"$in": WATER_BREEDS},
            COL_AGE_WEEKS: {"$gte": 26, "$lte": 156},
            # preference specified as Intact Female - we try to match 'Intact Female' substring
            COL_SEX: {"$regex": "Intact Female", "$options": "i"}
        }
    elif filter_type == 'mountain':
        query = {
            COL_BREED: {"$in": MOUNTAIN_BREEDS},
            COL_AGE_WEEKS: {"$gte": 26, "$lte": 156},
            COL_SEX: {"$regex": "Intact Male", "$options": "i"}
        }
    elif filter_type == 'disaster':
        query = {
            COL_BREED: {"$in": DISASTER_BREEDS},
            COL_AGE_WEEKS: {"$gte": 20, "$lte": 300},
            COL_SEX: {"$regex": "Intact Male", "$options": "i"}
        }
    else:  # reset or unknown
        query = {}
    return query

@app.callback(
    Output('datatable-id', 'data'),
    Output('datatable-id', 'columns'),
    Input('apply-filter', 'n_clicks'),
    State('filter-type', 'value')
)
def update_dashboard(n_clicks, filter_type):
    """
    Run the MongoDB read based on the selected filter, then return table data + columns.
    """
    query = build_query_for_filter(filter_type)
    try:
        recs = list(db.read(query))
    except Exception as e:
        # fail-safe: return empty list and keep previous columns
        recs = []
        print("DB read error:", e)

    if len(recs) == 0:
        # Return empty data but columns based on df to avoid UI crash
        cols = [{"name": i, "id": i} for i in df.columns]
        return [], cols

    df2 = pd.DataFrame.from_records(recs)
    if '_id' in df2.columns:
        df2.drop(columns=['_id'], inplace=True)
    # Ensure columns reflect df2 columns
    cols = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df2.columns]
    return df2.to_dict('records'), cols

###############################
# Graph update from table data
###############################
@app.callback(
    Output('graph-id', "children"),
    Input('datatable-id', "derived_virtual_data")
)
def update_graphs(viewData):
    # viewData is the current table view as list-of-dicts
    if not viewData:
        # show a message if no data
        return html.Div("No data to display for chart.", style={'padding':'20px'})

    dff = pd.DataFrame.from_dict(viewData)
    # fallback if breed column missing
    if COL_BREED not in dff.columns:
        return html.Div("Breed field not found in current data.", style={'padding':'20px'})

    # compute breed counts (top 8)
    counts = dff[COL_BREED].fillna("Unknown").value_counts().nlargest(8)
    fig = px.pie(names=counts.index, values=counts.values, title="Top Breeds in Current View")
    return dcc.Graph(figure=fig)

###############################
# Map update from table data
###############################
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    if not viewData:
        return html.Div("No data to map.", style={'padding':'20px'})
    dff = pd.DataFrame.from_dict(viewData)

    # if no location columns -> notify
    if COL_LAT not in dff.columns or COL_LONG not in dff.columns:
        return html.Div("Latitude/Longitude columns not present.", style={'padding':'20px'})

    # if user has selected a row, show marker for that row; otherwise center map on Austin and show multiple markers (first 10)
    try:
        if selected_rows:
            row_idx = selected_rows[0]
            row = dff.iloc[row_idx]
            lat = float(row.get(COL_LAT, np.nan))
            lon = float(row.get(COL_LONG, np.nan))
            if np.isnan(lat) or np.isnan(lon):
                return html.Div("Selected row has no valid coordinates.", style={'padding':'20px'})

            return dl.Map(style={'width': '100%', 'height': '500px'}, center=[lat, lon], zoom=12, children=[
                dl.TileLayer(),
                dl.Marker(position=[lat, lon], children=[
                    dl.Tooltip(str(row.get(COL_BREED, 'Unknown'))),
                    dl.Popup([html.H4(str(row.get(COL_NAME, 'No name'))),
                              html.P(f"Breed: {row.get(COL_BREED, '')}"),
                              html.P(f"Age (weeks): {row.get(COL_AGE_WEEKS, '')}")])
                ])
            ])
        else:
            # show up to 30 markers from the current view
            markers = []
            for _, r in dff.head(30).iterrows():
                try:
                    lat = float(r.get(COL_LAT, np.nan))
                    lon = float(r.get(COL_LONG, np.nan))
                except:
                    continue
                if np.isnan(lat) or np.isnan(lon):
                    continue
                markers.append(dl.Marker(position=[lat, lon], children=[
                    dl.Tooltip(str(r.get(COL_BREED, 'Unknown'))),
                    dl.Popup([html.H4(str(r.get(COL_NAME, 'No name'))),
                              html.P(f"Breed: {r.get(COL_BREED, '')}"),
                              html.P(f"Age (weeks): {r.get(COL_AGE_WEEKS, '')}")])
                ]))

            # default center Austin if possible
            center = [30.75, -97.48]
            return dl.Map(style={'width': '100%', 'height': '500px'}, center=center, zoom=10, children=[dl.TileLayer()] + markers)
    except Exception as e:
        print("Map error:", e)
        return html.Div("Error rendering map.", style={'padding':'20px'})

###############################
# Style selected column highlight
###############################
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Run server
if __name__ == '__main__':
    app.run_server(mode='inline')